In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision.datasets import MNIST

In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from PIL import Image
import csv
import numpy as np
import matplotlib.pyplot as plt


In [40]:

import matplotlib.pyplot as plt
%matplotlib inline
import torchvision.transforms as transforms
from skimage.transform import resize
import imageio


In [41]:
from os import listdir
from os.path import isfile, join
import csv
mypath = "/Users/rushi/Downloads/SoML-50/train/minus/"
subt_train = [f for f in listdir(mypath) if isfile(join(mypath, f))]
mypath = "/Users/rushi/Downloads/SoML-50/train/plus/"
addn_train = [f for f in listdir(mypath) if isfile(join(mypath, f))]
mypath = "/Users/rushi/Downloads/SoML-50/train/times/"
mult_train = [f for f in listdir(mypath) if isfile(join(mypath, f))]
mypath = "/Users/rushi/Downloads/SoML-50/train/div/"
divn_train = [f for f in listdir(mypath) if isfile(join(mypath, f))]


mypath = "/Users/rushi/Downloads/SoML-50/eval/minus/"
subt_test = [f for f in listdir(mypath) if isfile(join(mypath, f))]
mypath = "/Users/rushi/Downloads/SoML-50/eval/plus/"
addn_test = [f for f in listdir(mypath) if isfile(join(mypath, f))]
mypath = "/Users/rushi/Downloads/SoML-50/eval/times/"
mult_test = [f for f in listdir(mypath) if isfile(join(mypath, f))]
mypath = "/Users/rushi/Downloads/SoML-50/eval/div/"
divn_test = [f for f in listdir(mypath) if isfile(join(mypath, f))]


In [42]:
## Subtraction -> 0 ; Addition -> 1; Multiplication -> 2; Division -> 3

rows = []

fields = ['File','Label']

for f in subt_train:
    rows.append(["/Users/rushi/Downloads/SoML-50/train/minus/" + f,'0'])
for f in addn_train:
    rows.append(["/Users/rushi/Downloads/SoML-50/train/plus/" + f,'1'])
for f in mult_train:
    rows.append(["/Users/rushi/Downloads/SoML-50/train/times/" + f,'2'])
for f in divn_train:
    rows.append(["/Users/rushi/Downloads/SoML-50/train/div/" + f,'3'])
for f in subt_test:
    rows.append(["/Users/rushi/Downloads/SoML-50/eval/minus/" + f,'0'])
for f in addn_test:
    rows.append(["/Users/rushi/Downloads/SoML-50/eval/plus/"+ f,'1'])
for f in mult_test:
    rows.append(["/Users/rushi/Downloads/SoML-50/eval/times/" + f,'2'])
for f in divn_test:
    rows.append(["/Users/rushi/Downloads/SoML-50/eval/div/" + f,'3'])
    
filename = "datavals.csv"
with open(filename,'w',newline = '') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields) 
    csvwriter.writerows(rows)


    

In [28]:
root = "/Users/rushi/Downloads/SoML-50/"
annotations = "/Users/rushi/Downloads/datavals.csv"

In [43]:
class ShravanDataset(Dataset):
    def __init__(self,root,annotations,transform=ToTensor()):
        self.root = root
        self.transform = transform
        rows = []
        with open(annotations,mode='r') as csvfile:
            csvreader = csv.reader(csvfile)
            fields = next(csvreader)
            for row in csvreader:
                rows.append(row)
        self.data = rows
        

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = self.data[idx][0]
        image = Image.open(image_path)
        image = ToTensor()(image).squeeze(0)
        image_new = image[0]*0.2989 + image[1]*0.5870 + image[2]*0.1140
        image_new = resize(image_new,(28,28),mode = 'constant')
        y = (int)(self.data[idx][1])
        return image_new,y

In [44]:
dataset = ShravanDataset(root,annotations)

In [45]:
#from torch.utils.data import random_split
#train_dataset, val_dataset = random_split(dataset, [2000,509])
#print(dataset[0])
batchsize = 40
#from torch.utils.data import random_split
train_loader = DataLoader(dataset,batchsize,shuffle=False)
#train_dataset, val_dataset = random_split(dataset, [60000,0])
batchsize = 100
#val_loader = DataLoader(val_dataset,batchsize,shuffle=False)
input_size = 28*28
num_classes = 4

In [48]:
class Net_OPER(nn.Module):   
    def __init__(self):
        super(Net_OPER, self).__init__()
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(1, 4, kernel_size=5, stride=1, padding=1),
            nn.BatchNorm2d(4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.linear = nn.Sequential(
            nn.Linear(4*6*6,num_classes),
            nn.Softmax(),
        )


    # Defining the forward pass    
    def forward(self, x):
        #print(x.shape)
        x = x.reshape(-1,1,28,28)
        #print(x.shape)
        x = self.cnn_layers(x)
        #print(x.shape)
        x = x.reshape(-1,4*6*6)
        x = self.linear(x)
        return x

    def training_step(self, batch):
        images,labels = batch
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    """
    def validation_step(self, batch):
        images,labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss, 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))
    """

model = Net_OPER()

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

#loss_fn = F.cross_entropy
"""
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)
    """

In [51]:
def fit(epochs, lr, model, train_loader,opt_func=torch.optim.SGD):
    optimizer = opt_func(model.parameters(), lr)
    history = []

    for epoch in range(epochs):

        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
    return history
"""
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)
"""
    

'\n        result = evaluate(model, val_loader)\n        model.epoch_end(epoch, result)\n        history.append(result)\n'

In [52]:
fit(100,0.01,model,train_loader)
torch.save(model.state_dict(),'Test_oper.pt')

/Users/rushi/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


[]